<a href="https://colab.research.google.com/github/ZS4MLDL/learn_pytorch/blob/main/Transformer_Scratch_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, randomsplit
from torch.utils.tensorboard import SummaryWriter
#Math
import math
#Hugging Face
from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevel
from tokenizers.pre_tokenizers import Whitespace

from pathlib import path
from tqdm import tqdm
from typing import Any
import warnings

!pip install datasets

In [ ]:
class InputEmbedding(nn.Module):
  def __init__(self, d_model:int, vocab_size:int):
    super().__init__()
    self.d_model = d_model
    self.vocab_size = vocab_size
    self.embedding = nn.Embedding(vocab_size, d_model)

    def forward(self, x):
      return self.embedding(x) * math.sqrt(self.d_model)

In [ ]:
class PositionalEncoding(nn.Module):
  def __init__(self, d_model:int, seq_len:int,drop_out:float):
    super().__init__()
    self.d_model = d_model
    self.seq_len = seq_len
    self.drop_out = nn.Dropout(drop_out)

    pe = torch.zero(seq_len, d_model)
    position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1) #[seq_len, 1]
    div_term = torch.exp(torch.arange(0,d_model,2).float() * (-math.log(10000.0)/d_model))
    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)
    pe = pe.unsqueeze(0)
    self.register_buffer('pe', pe)

  def forward(self, x):
    x = x+ (self.pe[:,:[x.shape[1],:]]).requires_grad(False)
    return self.dropout(x)


